# **Baseline Notebook**



---
## Setup Environment

In [1]:
# DO NOT MODIFY THE CODE IN THIS CELL
!pip install -q utstd

from utstd.folders import *
from utstd.ipyrenders import *

at = AtFolder(
    course_code=36106,
    assignment="AT3",
)
at.run()

import warnings
warnings.simplefilter(action='ignore')

ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'C:\\Users\\brohao\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~-0earn\\.libs\\msvcp140.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



You can now save your data files in: c:\Users\brohao\Desktop\UTS\36106\ClassificationAnalysis-36106-GroupAssignment\36106\assignment\AT3\data


---
## Student Information

In [2]:

group_name = "Group 12"
student_name = "Jiayu Hao"
student_id = "25948860"

In [3]:
# Do not modify this code
print_tile(size="h1", key='group_name', value=group_name)

In [4]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_name', value=student_name)

In [5]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_id', value=student_id)

---
## 0. Python Packages

### 0.a Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [6]:
! pip install numpy
! pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 0.b Import Packages

In [ ]:

import pandas as pd
import altair as alt

In [8]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report

---
## A. Assess Baseline Model

In [9]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Load data
try:
  X_train = pd.read_csv(at.folder_path / 'X_train.csv')
  y_train = pd.read_csv(at.folder_path / 'y_train.csv')

  X_val = pd.read_csv(at.folder_path / 'X_val.csv')
  y_val = pd.read_csv(at.folder_path / 'y_val.csv')

  X_test = pd.read_csv(at.folder_path / 'X_test.csv')
  y_test = pd.read_csv(at.folder_path / 'y_test.csv')
except Exception as e:
  print(e)

### A.1 Generate Predictions with Baseline Model

In [10]:
# dataset overview
print(X_train.head(5))
print(y_train.mean())

   customer_key  freq  qty_sum  n_products  revenue_sum   avg_price  \
0         20946     1        5           2     166.9600   41.740000   
1         14652     1        8           3     875.1243  175.024860   
2         28123     1        3           1       4.9900    4.990000   
3         13498     1        4           3     100.0900   33.363333   
4         28231     1        9           2      88.2500   17.650000   

   avg_margin  returns_total  age  annual_income  number_dependents  \
0    0.328998          371.0   46        30000.0                0.0   
1    0.597124          191.0   62        60000.0                1.0   
2    0.625992           69.0   47        40000.0                0.0   
3    0.625997          160.0   60        30000.0                4.0   
4    0.625983          443.0   66        80000.0                2.0   

       education_level      occupation marital_status homeowner  orders_2020  \
0      Partial College        Clerical              S         N   

In [11]:
# drop id-like column
id_like = ['customer_key']

drop_cols = list(set(id_like))
print("Drop columns:", drop_cols)

X_train = X_train.drop(columns=drop_cols, errors='ignore')
X_val   = X_val.drop(columns=drop_cols, errors='ignore')
X_test  = X_test.drop(columns=drop_cols, errors='ignore')

Drop columns: ['customer_key']


In [12]:
# numerical column and categorical column
num_cols = X_train.select_dtypes(include=['number','float','int','Int64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object','category','bool']).columns.tolist()

# sparse preparation (For LR) 
pre_sparse = ColumnTransformer([
    ('num', StandardScaler(with_mean=False), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), cat_cols)
], remainder='drop')

baseline_lr = Pipeline([
    ('prep', pre_sparse),
    ('clf', LogisticRegression(max_iter=2000, class_weight='balanced', solver='liblinear'))
])

# Fitting LR baseline
baseline_lr.fit(X_train, y_train)

p_val  = baseline_lr.predict_proba(X_val)[:,1]
p_test = baseline_lr.predict_proba(X_test)[:,1]

### A.2 Selection of Performance Metrics

> Provide some explanations on why you believe the performance metrics you chose is appropriate


In [13]:
print("VAL  ROC-AUC/PR-AUC:", round(roc_auc_score(y_val, p_val),3), round(average_precision_score(y_val, p_val),3))
print("TEST ROC-AUC/PR-AUC:", round(roc_auc_score(y_test, p_test),3), round(average_precision_score(y_test, p_test),3))

VAL  ROC-AUC/PR-AUC: 0.664 0.268
TEST ROC-AUC/PR-AUC: 0.649 0.234


In [14]:
# Provide some explanations on why you believe the performance metrics you chose is appropriate

performance_metrics_explanations = """
The selected metrics, ROC-AUC and PR-AUC, are appropriate because the 
dataset is imbalanced (only about 11% positive class), and these metrics 
evaluate model discrimination more reliably than accuracy. ROC-AUC measures 
overall separability between buyers and non-buyers, independent of any 
threshold, while PR-AUC focuses on the model’s ability to identify true 
buyers among the top-ranked predictions, directly reflecting marketing 
efficiency. Using these metrics allows us to fairly assess performance 
across all probability levels and later convert the ranking quality into 
business value through cost–benefit threshold tuning.
"""

In [15]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='performance_metrics_explanations', value=performance_metrics_explanations)

### A.3 Baseline Model Performance

> Provide some explanations on model performance


In [16]:
# Simple threshold ( = 0.5) compare with optimal threshold

# def standard of gain function
W_TP, W_FP, W_FN, W_TN = 100, -20, -80, 0  
# choose threshold
def pick_best_threshold(y_true, p):
    thrs = np.linspace(0.01,0.99,99)
    best_t, best_gain = 0.5, -1e18
    for t in thrs:
        pred = (p>=t).astype(int)
        TP = ((pred==1)&(y_true==1)).sum()
        FP = ((pred==1)&(y_true==0)).sum()
        FN = ((pred==0)&(y_true==1)).sum()
        TN = ((pred==0)&(y_true==0)).sum()
        gain = TP*W_TP + FP*W_FP + FN*W_FN + TN*W_TN
        if gain > best_gain:
            best_gain, best_t = gain, t
    return best_t, best_gain

best_thr, best_gain = pick_best_threshold(y_val.values, p_val)
print(f"Best threshold on VAL: {best_thr:.2f} | Expected gain: {best_gain:.0f}")
# plot performance
def show_eval(tag, y, p, thr):
    pred = (p>=thr).astype(int)
    print(f"\n{tag} @thr={thr:.2f}")
    print(confusion_matrix(y, pred))
    print(classification_report(y, pred, digits=3))

show_eval("TEST (0.50)", y_test.values, p_test, 0.50)
show_eval("TEST (best_thr)", y_test.values, p_test, best_thr)

Best threshold on VAL: 0.01 | Expected gain: -3932100

TEST (0.50) @thr=0.50
[[444 241]
 [ 40  46]]
              precision    recall  f1-score   support

           0      0.917     0.648     0.760       685
           1      0.160     0.535     0.247        86

    accuracy                          0.636       771
   macro avg      0.539     0.592     0.503       771
weighted avg      0.833     0.636     0.702       771


TEST (best_thr) @thr=0.01
[[  0 685]
 [  0  86]]
              precision    recall  f1-score   support

           0      0.000     0.000     0.000       685
           1      0.112     1.000     0.201        86

    accuracy                          0.112       771
   macro avg      0.056     0.500     0.100       771
weighted avg      0.012     0.112     0.022       771



In [17]:
# Provide some explanations on model performance
baseline_performance_explanations = """
The baseline logistic regression shows moderate performance (ROC-AUC = 0.65, 
PR-AUC = 0.23), meaning it can separate buyers from non-buyers better than 
random but still misses many true positives. At the default threshold (0.5), 
it achieves good precision but low recall; 
the optimal threshold (0.01) gives unrealistic all-positive predictions 
because the model’s probabilities are poorly calibrated. The large negative 
expected gain reflects this imbalance and the unscaled cost weights. 
Overall, the model is a reasonable interpretative baseline but needs 
nonlinear and calibrated methods to improve decision quality.
"""

In [18]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='baseline_performance_explanations', value=baseline_performance_explanations)